In [147]:
import pandas as pd
from fpdf import FPDF
import yfinance as yf

# Load the Excel file
df_xlsx = pd.read_excel("Book2.xlsx")

# Filter buy and sell data
data_for_buy = df_xlsx[df_xlsx['TYPE'] == 'Buy']
data_for_sell = df_xlsx[df_xlsx['TYPE'] == 'Sell']

# Summarize the quantities by ticker
buy_summary = data_for_buy.groupby('EXCHANGE CODE: TICKER')['UNITS'].sum().to_dict()
sell_summary = data_for_sell.groupby('EXCHANGE CODE: TICKER')['UNITS'].sum().to_dict()
buy_price_dict = data_for_buy.groupby('EXCHANGE CODE: TICKER')['AMOUNT'].sum().to_dict()

sell_price_dict = data_for_sell.groupby('EXCHANGE CODE: TICKER')['AMOUNT'].sum().to_dict()

# Function to fetch the current stock price using yfinance for Indian stocks
def get_stock_price(ticker):
    try:
        # Assuming NSE symbols, append '.NS' to the ticker. Adjust if using BSE.
        stock = yf.Ticker(f"{ticker[4:]}.NS")  # Use '.BO' for BSE
        price = stock.history(period="1d")['Close'].iloc[-1]  # Get the latest closing price
        return round(float(price), 2)
    except Exception as e:
        print(f"Error fetching price for {ticker[4:]}: {e}")
        return None

net_quant = {key: round((buy_summary[key] - sell_summary[key]), 5) for key in sell_summary if round((buy_summary[key] - sell_summary[key]), 5) != 0}
difference_dict = {key: ((buy_price_dict[key]-sell_price_dict[key])/net_quant[key]) for key in net_quant}

# Calculate the remaining stock and get the current price
result = {}
for key in difference_dict:
    buy_quantity = buy_summary.get(key, 0)
    sell_quantity = sell_summary[key]
    remaining_quantity = buy_quantity - sell_quantity
    
    # Only process if remaining quantity is greater than zero
    if remaining_quantity != 0:
        buy_price = difference_dict.get(key, 0)
        current_price = get_stock_price(key)
        
        if current_price is not None:
            current_value = round(remaining_quantity * current_price, 2)
            unrealized_profit = (buy_price*net_quant[key])-current_value
            total_cost_of_remaining_shares = buy_price * remaining_quantity
            profit_percentage = (unrealized_profit / buy_price*net_quant[key]) * 100 
            result[key] = [
                round(remaining_quantity, 10),  # Remaining quantity
                current_price,  # Current price of the stock
                buy_price, #price at which the stock was bought
                round(unrealized_profit, 2),  # Unrealized profit
                current_value,  # Current value of the portfolio/investment
                round(profit_percentage, 2)  # Profit percentage
            ]

# Print the result
print(result)


{'NSE:ABBOTINDIA': [0.68, 27309.85, 19403.41176470588, -5376.38, 18570.7, -18.84], 'NSE:BAJEL': [46.35, 271.3, 206.11823085221144, -3021.17, 12574.75, -67937.33], 'NSE:BBTC': [-0.005, 2242.55, 2425791.0000000005, -12117.75, -11.21, 0.0], 'NSE:BIKAJI': [30.16, 849.7, 411.90820954907156, -13203.8, 25626.95, -96678.47], 'NSE:HCLTECH': [14.04, 1592.65, 1520.0569800569801, -1019.21, 22360.81, -941.39], 'NSE:MAYURUNIQ': [24.32, 647.9, 508.91077302631584, -3380.22, 15756.93, -16153.51], 'NSE:METROPOLIS': [7.13, 2010.55, 988.7130434782608, -7285.7, 14335.22, -5254.0], 'NSE:MSUMI': [257.97, 69.9, 55.73838043183316, -3653.27, 18032.1, -1690817.09], 'NSE:PRAJIND': [20.76, 736.4, 357.31791907514446, -7869.74, 15287.66, -45722.81], 'NSE:SRF': [6.22, 2521.05, 2735.2250803858524, 1332.17, 15680.93, 302.94], 'NSE:WABAG': [8.31, 1300.6, 599.6594464500604, -5824.82, 10807.99, -8071.96], 'NSE:WIPRO': [29.22, 490.5, 344.42813141683774, -4268.22, 14332.41, -36209.99], 'NSE:YESBANK': [608.24, 24.11, 18.9367

In [114]:
pdf = FPDF()
pdf.set_auto_page_break(auto=True, margin=0)
pdf.add_page()

# Add a background color for the title
pdf.set_fill_color(200, 220, 255)
pdf.set_font("Arial", "B", 18)
pdf.cell(0, 12, txt="Stock Portfolio Summary", ln=True, align="C", fill=True)
pdf.ln(10)
pdf.set_right_margin(40)
# Set column headers with a background color and border
pdf.set_fill_color(169, 169, 169)  # Dark gray
pdf.set_text_color(255, 255, 255)  # White text for headers
pdf.set_font("Arial", "B", 12)
pdf.cell(38, 10, txt="Stock", border=1, align="C", fill=True)
pdf.cell(38, 10, txt="Net Quantity", border=1, align="C", fill=True)
pdf.cell(28, 10, txt="Current Price", border=1, align="C", fill=True)
pdf.cell(28, 10, txt="Profit/Loss", border=1, align="C", fill=True)
pdf.cell(28, 10, txt="Profit/Loss%", border=1, align="C", fill=True)
pdf.cell(38, 10, txt="Current Value", border=1, align="C", fill=True)
pdf.ln()

# Add data rows
pdf.set_font("Arial", "", 12)
pdf.set_text_color(0, 0, 0)  # Black text for data

for idx, (key, value) in enumerate(result.items()):
    # Alternate row background color for better readability
    if idx % 2 == 0:
        pdf.set_fill_color(245, 245, 245)  # Light gray
        fill = True
    else:
        fill = False

    # Add stock name and quantities
    pdf.cell(38, 10, txt=key[4:], border=1, align="C", fill=fill)  # Stock name without the exchange prefix
    pdf.cell(38, 10, txt=str(value[0]), border=1, align="C", fill=fill)  # Total quantities
    
    # Add current price
    pdf.cell(28, 10, txt=str(value[1]), border=1, align="C", fill=fill)  # Current price

    # Add profit with color coding
    if value[2] == 0:
        pdf.set_text_color(0, 128, 0)  # Green for profit
    else:
        pdf.set_text_color(255, 0, 0)  # Red for loss
    pdf.cell(28, 10, txt=str(value[2]), border=1, align="C", fill=fill)  # Profit
    pdf.set_text_color(0, 0, 0)  # Reset text color to black
    
    
    # Add profit percentage and current value
    pdf.cell(28, 10, txt=str(value[4]) + '%', border=1, align="C", fill=fill)  # Profit percentage
    pdf.cell(38, 10, txt=str(value[3]), border=1, align="C", fill=fill)  # Current value
    pdf.ln()

# Add a footer
pdf.set_y(-15)
pdf.set_font("Arial", "I", 8)
pdf.set_text_color(128, 128, 128)
pdf.cell(0, 10, txt="End of Report", align="C")

# Save the PDF to a file
pdf.output("stock_portfolio_summary.pdf")

print("PDF created successfully.")

PDF created successfully.


In [128]:
from fpdf import FPDF

pdf = FPDF()
pdf.set_auto_page_break(auto=True, margin=10)
pdf.add_page()

# Add a background color for the title
pdf.set_fill_color(200, 220, 255)
pdf.set_font("Arial", "B", 12)  # Smaller font size for the title
pdf.cell(0, 12, txt="Stock Portfolio Summary", ln=True, align="C", fill=True)
pdf.ln(10)
pdf.set_right_margin(10)

# Set column headers with a background color and border
pdf.set_fill_color(169, 169, 169)  # Dark gray
pdf.set_text_color(255, 255, 255)  # White text for headers
pdf.set_font("Arial", "B", 10)  # Smaller font size for headers

# Adjust the width of each column so that it fits within the page
pdf.cell(25, 10, txt="Stock", border=1, align="C", fill=True)
pdf.cell(30, 10, txt="Net Quantity", border=1, align="C", fill=True)
pdf.cell(20, 10, txt="Buy Price", border=1, align="C", fill=True)
pdf.cell(25, 10, txt="Current Price", border=1, align="C", fill=True)
pdf.cell(25, 10, txt="Profit/Loss", border=1, align="C", fill=True)
pdf.cell(30, 10, txt="Profit/Loss%", border=1, align="C", fill=True)
pdf.cell(35, 10, txt="Current Value", border=1, align="C", fill=True)
pdf.ln()

# Add data rows
pdf.set_font("Arial", "", 10)  # Smaller font size for data rows
pdf.set_text_color(0, 0, 0)  # Black text for data

for idx, (key, value) in enumerate(result.items()):
    # Alternate row background color for better readability
    if idx % 2 == 0:
        pdf.set_fill_color(245, 245, 245)  # Light gray
        fill = True
    else:
        fill = False

    # Add stock name and quantities
    pdf.cell(25, 10, txt=key[4:], border=1, align="C", fill=fill)  # Stock name without the exchange prefix
    pdf.cell(30, 10, txt=str(value[0]), border=1, align="C", fill=fill)  # Total quantities
    
    # Add buy price
    pdf.cell(20, 10, txt=str(value[2]), border=1, align="C", fill=fill)  # Buy price
    
    # Add current price
    pdf.cell(25, 10, txt=str(value[1]), border=1, align="C", fill=fill)  # Current price

    # Add profit with color coding
    if value[3] >= 0:
        pdf.set_text_color(0, 128, 0)  # Green for profit
    else:
        pdf.set_text_color(255, 0, 0)  # Red for loss
    pdf.cell(25, 10, txt=str(value[3]), border=1, align="C", fill=fill)  # Profit/Loss
    pdf.set_text_color(0, 0, 0)  # Reset text color to black
    
    # Add profit percentage and current value
    pdf.cell(30, 10, txt=str(value[5]) + '%', border=1, align="C", fill=fill)  # Profit percentage
    pdf.cell(35, 10, txt=str(value[4]), border=1, align="C", fill=fill)  # Current value
    pdf.ln()

# Add a footer
pdf.set_y(-15)
pdf.set_font("Arial", "I", 8)  # Footer font size
pdf.set_text_color(128, 128, 128)
pdf.cell(0, 10, txt="End of Report", align="C")

# Save the PDF to a file
pdf.output("stock_portfolio.pdf")

print("PDF created successfully.")


PDF created successfully.
